In [60]:
import nltk
#nltk.download()
import string
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
import sklearn.datasets as sk
import matplotlib.pyplot as plt

In [61]:
stopWords=(stopwords.words('english'))

In [71]:
def cleanText(text):
    text = str(text)
    if text.startswith("b\"") or text.startswith("b\'"):
        text = text[1:]
    text = text.replace("\"", "").replace("\'", "")
    # lower case
    text = text.lower()
    # tokenize
    filtered_sentence = word_tokenize(text)
    # # remove stop words
    # filtered_sentence = [w for w in tokenized if not w in stopWords]
    # filtered_sentence = [w for w in filtered_sentence if len(w)>1]
    # stemming
    ps = PorterStemmer()
    for i in range(len(filtered_sentence)-1):
        try:
            filtered_sentence[i] = ps.stem(filtered_sentence[i])
        except Exception as inst:
            filtered_sentence[i]=filtered_sentence[i]
    # remove stop words after stem
    filtered_sentence = [w for w in filtered_sentence if not w in stopWords]
    filtered_sentence = [w for w in filtered_sentence if len(w)>1]
    text= " ".join(filtered_sentence)
    return text.strip()

In [72]:
combined_news_djia_df = pd.read_csv( "C:\\Users\\ravedan\\PycharmProjects\\Project-Stock-Market-Prediction\\Data\\Combined_News_DJIA.csv")

In [73]:
headlines_columns = combined_news_djia_df.columns[range(2, 27)]
combined_news_djia_df[headlines_columns] = combined_news_djia_df[headlines_columns].applymap(cleanText)

In [74]:
samples_per_class = []
for c in [0, 1]:
    samples_per_class.append([c, combined_news_djia_df["Label"].value_counts()[c]])
pd.DataFrame(samples_per_class, columns=['Class', '# of Samples'])

,Class,# of Samples
0,0,924
1,1,1065


In [95]:
n = 10
for c in [0, 1]:
    print 'Class:', c
    class_freq = pd.Series()
    class_df = combined_news_djia_df.loc[combined_news_djia_df["Label"] == c]
    for headline_column in combined_news_djia_df[headlines_columns]:
        class_freq = class_freq.append(pd.Series(" ".join(class_df[headline_column]).split()).value_counts()[:n])
    class_freq = class_freq.groupby(by=class_freq.index).sum().sort_values(ascending=False,axis=0)
    print(class_freq.to_frame('freq'))


Class: 0
         freq
ha       2190
say      1591
us       1287
wa       1153
kill     1102
world     932
new       914
year      833
govern    531
china     522
israel    490
peopl     384
polic     310
hi        238
russia    227
countri   196
attack    160
war        44
u.s.       42
report     41
first      41
thi        40
state      39
iran       39
Class: 1


         freq
ha       2495
say      1860
us       1471
kill     1363
wa       1234
new       996
world     926
china     895
year      829
israel    719
govern    480
polic     291
peopl     238
russia    195
attack    192
hi        187
war       138
use        98
countri    91
iran       86
ban        56
drug       55
protest    53
call       49
state      46
one        42
